In [1]:
!nvidia-smi


Thu Dec 26 19:24:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              1W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# !pip install ultralytics==8.0.20

from IPython import display
# display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.49  Python-3.12.6 torch-2.6.0.dev20241112+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Setup complete  (12 CPUs, 15.3 GB RAM, 213.0/476.0 GB disk)


In [3]:
from ultralytics import YOLO

from IPython.display import display, Image

In [4]:
import os
import random
import shutil
from math import floor

def create_dirs(base_path, dirs):
    """
    Create directories if they do not exist.
    """
    for dir_name in dirs:
        dir_path = os.path.join(base_path, dir_name)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

def copy_files(files, src_dir, img_dest_dir, lbl_dest_dir):
    """
    Copy image and corresponding label file to the destination directories.
    Args:
        files (list): List of image file names.
        src_dir (str): Source directory where the files are located.
        img_dest_dir (str): Destination directory for images.
        lbl_dest_dir (str): Destination directory for labels.
    """
    for file in files:
        image_file = os.path.join(src_dir, file)
        label_file = os.path.join(src_dir, file.replace('.jpg', '.txt')) 
        
        # Copy the image file
        shutil.copy(image_file, os.path.join(img_dest_dir, file))
        
        # Copy the corresponding label file if it exists
        if os.path.exists(label_file):
            shutil.copy(label_file, os.path.join(lbl_dest_dir, file.replace('.jpg', '.txt')))

def split_dataset(dataset_dir, output_dir, train_ratio=0.7, valid_ratio=0.15, test_ratio=0.15):
   
    # Get list of all image files in the dataset directory
    all_files = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg')]
    random.shuffle(all_files)  # Shuffle the dataset

    total_files = len(all_files)
    train_size = floor(total_files * train_ratio)
    valid_size = floor(total_files * valid_ratio)
    test_size = total_files - train_size - valid_size

    train_files = all_files[:train_size]
    valid_files = all_files[train_size:train_size + valid_size]
    test_files = all_files[train_size + valid_size:]

    create_dirs(output_dir, ['train/images', 'train/labels', 'valid/images', 'valid/labels', 'test/images', 'test/labels'])
    copy_files(train_files, dataset_dir, os.path.join(output_dir, 'train/images'), os.path.join(output_dir, 'train/labels'))
    copy_files(valid_files, dataset_dir, os.path.join(output_dir, 'valid/images'), os.path.join(output_dir, 'valid/labels'))
    copy_files(test_files, dataset_dir, os.path.join(output_dir, 'test/images'), os.path.join(output_dir, 'test/labels'))

    print(f"Dataset split complete. Train: {train_size}, Valid: {valid_size}, Test: {test_size}")
# calling func
dataset_dir = 'C:/project_7th/Accident-Detection-Model/data'
output_dir = 'C:/project_7th/Accident-Detection-Model/dataset'
split_dataset(dataset_dir, output_dir)


Dataset split complete. Train: 2815, Valid: 603, Test: 604


In [4]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available

True


In [5]:
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the first GPU


1
NVIDIA GeForce GTX 1650


In [7]:
# !yolo task=detect mode=train model=yolov8s.pt data= data.yaml epochs=10 imgsz=640 plots=True device=0

# Initialize the model (YOLOv8 - you can specify a smaller version such as 'yolov8n.pt' for faster training)
model = YOLO('yolov8n.pt')  # You can also use 'yolov8.pt' for the base model

# Train the model on your custom dataset
model.train(data='data.yaml', epochs=200, imgsz=640, batch=16, val=True,plots=True, device=0)

New https://pypi.org/project/ultralytics/8.3.55 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.49  Python-3.12.6 torch-2.6.0.dev20241112+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

train: Scanning C:\project_7th\yolov8\train\labels... 2279 images, 28 backgrounds, 0 corrupt: 100%|██████████| 2279/2279 [00:02<00:00, 856.39it/s]


train: New cache created: C:\project_7th\yolov8\train\labels.cache


val: Scanning C:\project_7th\yolov8\valid\labels... 301 images, 2 backgrounds, 0 corrupt: 100%|██████████| 301/301 [00:00<00:00, 482.60it/s]


val: New cache created: C:\project_7th\yolov8\valid\labels.cache
Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train4
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        301       1768      0.586      0.618      0.597      0.375



200 epochs completed in 3.898 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 6.3MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.49  Python-3.12.6 torch-2.6.0.dev20241112+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.48it/s]


                   all        301       1768      0.598        0.6      0.629      0.385
            Motorcycle         47        149      0.521      0.443      0.507      0.204
                 Truck         80        176      0.708      0.722      0.739      0.393
          car-accident         63         66      0.541      0.642      0.609      0.487
   car-severe-accident        113        121      0.764      0.736      0.809       0.59
                  cars        161       1198      0.734      0.694      0.722      0.332
   motorcycle-accident         22         23      0.309      0.565      0.512      0.353
motorcycle-severe-accident         29         35      0.613        0.4      0.503      0.338
Speed: 0.4ms preprocess, 5.9ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\detect\train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E0F0E5C260>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [9]:
!yolo task=detect mode=val model=runs/detect/train4/weights/best.pt data=data.yaml save=true

Ultralytics 8.3.49 🚀 Python-3.12.6 torch-2.6.0.dev20241112+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs
                   all        301       1768      0.597      0.599      0.627      0.385
            Motorcycle         47        149      0.518      0.443      0.501      0.201
                 Truck         80        176      0.694      0.716      0.731       0.39
          car-accident         63         66       0.54      0.641      0.609      0.486
   car-severe-accident        113        121      0.764      0.736      0.809       0.59
                  cars        161       1198      0.732      0.689      0.723      0.334
   motorcycle-accident         22         23      0.318      0.565      0.515      0.354
motorcycle-severe-accident         29         35      0.613        0.4      0.504      0.337
Speed: 0.7ms preprocess, 7.1ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved 


val: Scanning C:\project_7th\yolov8\valid\labels.cache... 301 images, 2 backgrounds, 0 corrupt: 100%|██████████| 301/301 [00:00<?, ?it/s]
val: Scanning C:\project_7th\yolov8\valid\labels.cache... 301 images, 2 backgrounds, 0 corrupt: 100%|██████████| 301/301 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/19 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   5%|▌         | 1/19 [00:00<00:09,  1.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  11%|█         | 2/19 [00:00<00:06,  2.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  16%|█▌        | 3/19 [00:01<00:07,  2.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  21%|██        | 4/19 [00:01<00:06,  2.40it/s]
                 Class     Images  Instances 

In [10]:
import cv2
from ultralytics import YOLO

def detect_objects(image, model, conf_thres=0.6):
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(img_rgb)

    # Access detections from results
    detections = results[0].boxes

    class_ids, confidences, boxes = [], [], []

    # Iterate over detections and filter based on confidence
    for detection in detections:
        # Extract bounding box coordinates, confidence, and class ID
        x1, y1, x2, y2 = detection.xyxy[0].cpu().numpy().astype(int)
        conf = detection.conf[0].item()  # Convert confidence to Python float
        class_id = int(detection.cls[0].item())  # Convert class_id to Python int

        if conf > conf_thres:
            class_ids.append(class_id)
            confidences.append(conf)
            boxes.append([x1, y1, x2 - x1, y2 - y1])  # Convert (x1, y1, x2, y2) to (x, y, width, height)

    return class_ids, confidences, boxes

def draw_boxes(image, class_ids, confidences, boxes):
    for i, box in enumerate(boxes):
        x, y, w, h = box
        conf = confidences[i]
        class_id = class_ids[i]

        # Draw the bounding box
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Create the label with class ID and confidence
        label = f"ID: {class_id}, Conf: {conf:.2f}"

        # Get the label size and adjust position
        (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        label_position = (x, y - 10 if y - 10 > 10 else y + 10)  # Ensure label is within frame bounds

        # Draw a filled rectangle behind the text for better readability
        cv2.rectangle(image, (x, label_position[1] - label_height), (x + label_width, label_position[1]), (0, 255, 0), cv2.FILLED)

        # Put the label text above the bounding box
        cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    return image

model_path = "C:/project_7th/yolov8/runs/detect/train2/weights/best.pt"
model = YOLO(model_path)

# Open video file
video_path = "C:/project_7th/yolov8/test_video_new.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties for writing the output video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_path = "C:/project_7th/yolov8/output_video.mp4"

# Define the and create VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run object detection
    class_ids, confidences, boxes = detect_objects(frame, model)

    # Draw the bounding boxes
    frame_with_boxes = draw_boxes(frame, class_ids, confidences, boxes)

    # Write the processed frame to the output video
    out.write(frame_with_boxes)

    # Optionally display the video in real-time (press 'q' to stop)
    cv2.imshow('YOLOv8 Detection', frame_with_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video objects
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 Accident, 61.6ms
Speed: 12.7ms preprocess, 61.6ms inference, 12.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Accident, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 2.7ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 1.3ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.4ms
Speed: 2.2ms preprocess, 10.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.0ms
Speed: 2.2ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.3ms
Speed: 1.8ms preprocess, 8.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.1ms
Speed: 2.2ms preprocess, 9.1ms inference, 1.0ms po

In [11]:
import cv2
from ultralytics import YOLO

# Define the class names as per your model
class_names = ['Motorcycle', 'Truck', 'car-accident', 'car-severe-accident', 'cars', 'motorcycle-accident', 'motorcycle-severe-accident']

def detect_objects(image, model, conf_thres=0.6):
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(img_rgb)

    # Access detections from results
    detections = results[0].boxes

    class_ids, confidences, boxes = [], [], []

    # Iterate over detections and filter based on confidence
    for detection in detections:
        # Extract bounding box coordinates, confidence, and class ID
        x1, y1, x2, y2 = detection.xyxy[0].cpu().numpy().astype(int)
        conf = detection.conf[0].item()  # Convert confidence to Python float
        class_id = int(detection.cls[0].item())  # Convert class_id to Python int

        if conf > conf_thres:
            class_ids.append(class_id)
            confidences.append(conf)
            boxes.append([x1, y1, x2 - x1, y2 - y1])  # Convert (x1, y1, x2, y2) to (x, y, width, height)

    return class_ids, confidences, boxes

def draw_boxes(image, class_ids, confidences, boxes):
    for i, box in enumerate(boxes):
        x, y, w, h = box
        conf = confidences[i]
        class_id = class_ids[i]

        # Get the class name for the detected class ID
        class_name = class_names[class_id]

        # Draw the bounding box
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Create the label with class name and confidence
        label = f"{class_name}, Conf: {conf:.2f}"

        # Get the label size and adjust position
        (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        label_position = (x, y - 10 if y - 10 > 10 else y + 10)  # Ensure label is within frame bounds

        # Draw a filled rectangle behind the text for better readability
        cv2.rectangle(image, (x, label_position[1] - label_height), (x + label_width, label_position[1]), (0, 255, 0), cv2.FILLED)

        # Put the label text above the bounding box
        cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    return image

model_path = "C:/project_7th/yolov8/runs/detect/train/weights/best.pt"
model = YOLO(model_path)

# Open video file
video_path = "C:/project_7th/yolov8/test_video_new.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties for writing the output video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_path = "C:/project_7th/yolov8/output_video.mp4"

# Define the and create VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run object detection
    class_ids, confidences, boxes = detect_objects(frame, model)

    # Draw the bounding boxes
    frame_with_boxes = draw_boxes(frame, class_ids, confidences, boxes)

    # Write the processed frame to the output video
    out.write(frame_with_boxes)

    # Optionally display the video in real-time (press 'q' to stop)
    cv2.imshow('YOLOv8 Detection', frame_with_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video objects
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 cars, 23.4ms
Speed: 6.2ms preprocess, 23.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.4ms
Speed: 1.8ms preprocess, 10.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.4ms
Speed: 2.1ms preprocess, 9.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.8ms
Speed: 1.2ms preprocess, 7.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.6ms
Speed: 1.2ms preprocess, 7.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.7ms
Speed: 1.0ms preprocess, 7.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.5ms
Speed: 1.4ms preprocess, 7.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cars, 8.2ms
Speed: 1.0ms preprocess, 8.2ms inference, 2.0ms postprocess pe